In [51]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OrdinalEncoder

# Preprocessing the data and training the model

In [52]:
raw_dataset = pd.read_csv('../data/okcupid.csv') 
okcupid_profiles = raw_dataset.drop(columns="Unnamed: 0") 

As we can see from the output below, almost every column contains object types, which we can not use to fit the Random Forest. 
We need to convert the objects into numbers, and we can do that using OrdinalEncoder from sklearn.
We need to manage the missing data first tho

In [53]:
okcupid_profiles.dtypes

age              int64
status          object
sex             object
orientation     object
body_type       object
diet            object
drinks          object
drugs           object
education       object
ethnicity       object
height         float64
income           int64
job             object
last_online     object
location        object
offspring       object
pets            object
religion        object
sign            object
smokes          object
speaks          object
dtype: object

## Filling the missing data

The columns containing missing data are the following:

In [54]:
print(okcupid_profiles.isna().sum())

age                0
status             0
sex                0
orientation        0
body_type       5296
diet           24395
drinks          2985
drugs          14080
education       6628
ethnicity       5680
height             3
income             0
job             8198
last_online        0
location           0
offspring      35561
pets           19921
religion       20226
sign           11056
smokes          5512
speaks            50
dtype: int64


Comparing the missing data output with the dtype output, we can easily see how, except for height, all the missing data are categorical strings.

Since there are only three rows with missing values for height, instead of replacing the NaN with something like 0 or -1, or the average height, we think it's better to just drop them, since it is such a small number

In [55]:
okcupid_profiles = okcupid_profiles.dropna(how = 'any', subset = 'height') 

For all the others attributes, we will just replace the missing values with the 'missing' string.

In [56]:
okcupid_profiles = okcupid_profiles.fillna(value = 'missing')

And now all the columns contain something

In [57]:
print(okcupid_profiles.isna().sum())

age            0
status         0
sex            0
orientation    0
body_type      0
diet           0
drinks         0
drugs          0
education      0
ethnicity      0
height         0
income         0
job            0
last_online    0
location       0
offspring      0
pets           0
religion       0
sign           0
smokes         0
speaks         0
dtype: int64


## Encoding the data

In [58]:
enc = OrdinalEncoder()
enc.fit(okcupid_profiles)

OrdinalEncoder()

In [59]:
transformed_data = enc.transform(okcupid_profiles)

Now we have a Numpy array with the encoded data, so no more objects, but only numbers.

In [60]:
transformed_data.dtype

dtype('float64')

Let's convert the Numpy array to a Pandas dataframe

In [61]:
df_transformed = pd.DataFrame(transformed_data, 
                              columns = okcupid_profiles.columns)

In [62]:
df_transformed.head()

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,income,job,last_online,location,offspring,pets,religion,sign,smokes,speaks
0,4.0,3.0,1.0,2.0,0.0,11.0,5.0,1.0,25.0,98.0,...,0.0,20.0,26756.0,174.0,2.0,14.0,3.0,16.0,2.0,1.0
1,17.0,3.0,1.0,2.0,2.0,7.0,3.0,3.0,31.0,216.0,...,7.0,8.0,28136.0,117.0,2.0,14.0,4.0,8.0,1.0,4596.0
2,20.0,0.0,1.0,2.0,11.0,0.0,5.0,0.0,12.0,189.0,...,0.0,12.0,24988.0,154.0,14.0,5.0,40.0,32.0,1.0,6334.0
3,5.0,3.0,1.0,2.0,11.0,18.0,5.0,0.0,25.0,216.0,...,1.0,19.0,26402.0,16.0,4.0,10.0,40.0,29.0,1.0,6481.0
4,11.0,3.0,1.0,2.0,1.0,3.0,5.0,1.0,9.0,22.0,...,0.0,0.0,25602.0,154.0,14.0,14.0,40.0,0.0,1.0,1.0


# Random Forest

## Income prediction model

In [63]:
y = df_transformed["income"]
X = df_transformed

# test_size = 0.3   means 70% training set | 30% test set
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 0.3, 
                                                    random_state = 42)

In [64]:
# n_estimators is the number of trees in the forest
rfc = RandomForestClassifier(n_estimators = 50)
rfc.fit(X_train, y_train)
rfc_prediction = rfc.predict(X_test)

In [65]:
print("Random Forest Classification report")
print(classification_report(y_test, rfc_prediction))
print("Random Forest Confusion Matrix")
print(confusion_matrix(y_test, rfc_prediction))

Random Forest Classification report
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     14491
         1.0       0.98      1.00      0.99       904
         2.0       0.93      0.98      0.96       332
         3.0       0.87      0.92      0.90       313
         4.0       0.84      0.89      0.87       299
         5.0       0.87      0.73      0.80       221
         6.0       0.94      0.84      0.89       213
         7.0       0.95      0.97      0.96       324
         8.0       0.94      0.99      0.96       469
         9.0       0.85      0.89      0.87       199
        10.0       0.87      0.36      0.51        36
        11.0       0.00      0.00      0.00        15
        12.0       0.90      0.82      0.86       167

    accuracy                           0.99     17983
   macro avg       0.84      0.80      0.81     17983
weighted avg       0.98      0.99      0.98     17983

Random Forest Confusion Matrix
[[14491     

C:\Users\meloncrush\Documents\python_projects\HR_ML_Project1\proj1VENV\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\meloncrush\Documents\python_projects\HR_ML_Project1\proj1VENV\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\meloncrush\Documents\python_projects\HR_ML_Project1\proj1VENV\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete

We can also get back to having string values using the inverse_transform() function

In [66]:
nparray_inverted = enc.inverse_transform(df_transformed)

In [67]:
okcupid_profiles = pd.DataFrame(nparray_inverted, columns = okcupid_profiles.columns)

In [68]:
okcupid_profiles.head()

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,income,job,last_online,location,offspring,pets,religion,sign,smokes,speaks
0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,"asian, white",...,-1,transportation,2012-06-28-20-30,"south san francisco, california","doesn't have kids, but might want them",likes dogs and likes cats,agnosticism and very serious about it,gemini,sometimes,english
1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,white,...,80000,hospitality / travel,2012-06-29-21-41,"oakland, california","doesn't have kids, but might want them",likes dogs and likes cats,agnosticism but not too serious about it,cancer,no,"english (fluently), spanish (poorly), french (..."
2,38,available,m,straight,thin,anything,socially,missing,graduated from masters program,missing,...,-1,missing,2012-06-27-09-10,"san francisco, california",missing,has cats,missing,pisces but it doesn&rsquo;t matter,no,"english, french, c++"
3,23,single,m,straight,thin,vegetarian,socially,missing,working on college/university,white,...,20000,student,2012-06-28-14-22,"berkeley, california",doesn't want kids,likes cats,missing,pisces,no,"english, german (poorly)"
4,29,single,m,straight,athletic,missing,socially,never,graduated from college/university,"asian, black, other",...,-1,artistic / musical / writer,2012-06-27-21-26,"san francisco, california",missing,likes dogs and likes cats,missing,aquarius,no,english
